## List comprehensions in Python

Python offers a convenient way to handle iterative structures (such as lists, set or dictionaries).  
It is considered good practice to be able to manipulate them fluently. Let's have a look why!

### Head-first into the topic!

In Python 3, the `range` keyword does not provide a list but a different structure that can easily be transformed into a list.

In [ ]:
range(10)

In [ ]:
type(range(10))  # this is not a list!

In [ ]:
list(range(10))  # but you can make a list out of it

The key point about this notation is performance: when you write `range(10000)`, you do not create a list of size 10000, but a structure that will give you a new integer each time you go through a loop.

In that perspective, you may be comfortable with the following snippet of code:

In [ ]:
for i in range(10):
    print(i, end=" ")

Now, let's say we want to compute $x \mapsto 2\cdot x$ for each element of a given list (resp. range).  
Let us compare the two following ways of writing it.

In [ ]:
a = range(1000000)

In [ ]:
%%time
new_list = [2 * x for x in a]

In [ ]:
%%time
new_list = []
for x in a:
    new_list.append(x)

<div class="alert alert-success">
<b>Definition:</b> Any syntax going like `[f(x) for x in array]` is referred to as <em>list comprehension</em>.  
</div>

<div class="alert alert-danger">
<b>Important note:</b> Besides being a more compact way to write things, list comprehension is also more efficient.
</div>

The official [docs.python.org](https://docs.python.org/) mentions it quickly [here](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions) but let's go through this more into details.

### List comprehensions are not only about lists

List comprehensions are based on a Python specific data structure called a generator.  
You can manipulate a generator with brackets (no bracket results in a `SyntaxError`).

In [ ]:
g = (str(i) for i in range(10))
type(g)

You can construct any data structure that accept *iterable structures* with a generator.

In [ ]:
list(g)

Note that the generator is now empty. You cannot build a second list from the same `g`:

In [ ]:
list(g)

So let's go through different constructions:
 - the regular Python `list`:

In [ ]:
# equivalent notations for lists
list(str(i) for i in range(10))
[str(i) for i in range(10)]

- the regular Python `set`:

In [ ]:
# equivalent notations for sets
set(str(i) for i in range(10))
{str(i) for i in range(10)}

- even dictionaries (keyword `dict`) with the column character `:`

In [ ]:
dict((i, str(i)) for i in range(10))
{i: str(i) for i in range(10)}

There are actually other useful constructions, e.g. `sorted` that builds a sorted list from a generator:

In [ ]:
[i * (-1) ** (i) for i in range(10)]

In [ ]:
sorted(i * (-1) ** (i) for i in range(10))

Building a `set` from the generator can also be useful as reflected on the following snippet:

In [ ]:
[i ** 2 for i in range(-5, 5)]

In [ ]:
{i ** 2 for i in range(-5, 5)}

You can also build there structures with more complex constructions.  
Recall the `enumerate` and/or `zip` constructions:

In [ ]:
french = ["un", "deux", "trois", "cat", "sank"]
english = ["one", "two", "three", "four", "five"]

words = {i + 1: f for i, f in enumerate(zip(french, english))}

In [ ]:
words

Looping on a dictionary iterates on the keys:

In [ ]:
[str(s) for s in words]

But you can also use the `dict.items()` method.  
Here is an example to show there is “no” limit in how you can use list comprehensions.

In [ ]:
[
    [key, value[0], len(value[0]), value[1].upper(), len(value[1])]
    for key, value in words.items()
]

### Common loop patterns using list comprehensions

Check the difference between a `zip` and a double loop construction:

In [ ]:
list(zip(french, english))

In [ ]:
[[(x, y) for x in french] for y in english]

In [ ]:
[(x, y) for x in french for y in english]

<div class="alert alert-danger">
<b>Important note:</b> Be aware of which of the two for loops iterates faster
</div>

You can also add conditions within the list comprehension:

In [ ]:
[a for a in range(10) if a % 2 == 0]

<div class="alert alert-warning">
<b>Exercice:</b> Use list comprehension to generate a list of $(a, b, c)$ integers such that $a^2 + b^2 = c^2$.
</div>

In [ ]:
[
    (a, b, c)
    for a in range(1, 30)
    for b in range(a, 30)
    for c in range(b, 30)
    if a ** 2 + b ** 2 == c ** 2
]

### Similar functions in numpy

Python loops are known to be expensive. List comprehensions help, but `numpy` takes a different approach by unfolding the loops using code written in C.

In [ ]:
import numpy as np

In [ ]:
%%time
new_list = [2 * x for x in range(1000000)]

In [ ]:
a = np.arange(1000000)

In [ ]:
%%time
new_list = 2 * a

<div class="alert alert-danger">
<b>Important note:</b> Although you cannot write as much in numpy as you can with list comprehension (because of the combination with the if construct), numpy remains the faster option.
</div>

* Nested comprehension may remind you the `meshgrid` function:

In [ ]:
z = [[(x + y) for x in range(10)] for y in range(5)]
z

In [ ]:
a, b = np.meshgrid(np.arange(10), np.arange(5))
c = a + b
c

* You can go through elements of a 2D array with the following constructs:

In [ ]:
[col[1] for col in z]

In [ ]:
c[:, 1]

Be careful though that `numpy` does not provide you a fresh copy of the array even if you create intermediate references.

In [ ]:
d = c[:, 0]
d *= 0
c

### Yet another implementation for this old academic problem

Say we want to get all the prime numbers lesser than n.  
List comprehension can be a comfortable way to compute the sieve of Eratosthenes.

In [ ]:
%%timeit
# First compute the non prime numbers
nope = [j for i in range(2, 8) for j in range(i * 2, 50, i)]
# Then build a new list not containing prime numbers
[x for x in range(2, 50) if x not in nope]

Actually the following implementation with `sets` may be more space and time efficient:

In [ ]:
%%timeit
sieve = set(range(2, 50))
for i in range(2, 8):
    sieve -= set(range(2 * i, 50, i))
sieve

### Another way to build your own generators

Let's recall the types of the structures we mentioned:

In [ ]:
type(range(10))

In [ ]:
type(i for i in range(10))

Python provides a `yield` keyword in order to let you write your own generator objects.  
When the program hits a yield keyword:

1. it *yields* the current value,
2. then waits for the next iteration in a loop (see the `__next__` keyword),
3. then re-enters the program where it was last interrupted. 

The [Syracuse suite](https://en.wikipedia.org/wiki/Collatz_conjecture) is a good study case for this programming style.  
See in the `doctest` how we build the suite using the `list` constructor on a `generator` structure.

In [ ]:
def syracuse(n):
    """Computes the Syracuse suite.

    >>> list(p for p in syracuse(28))
    [28, 14, 7, 22, 11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1]
    """
    assert n > 0
    yield n
    while n != 1:
        if n & 1 == 0:
            n = n // 2
        else:
            n = 3 * n + 1
        yield n

An interesting plot to draw is the length of the Syracuse list for all integers within a certain range.  
See how confortable it can be to write it.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def length(iterable):
    """Length of a generator.

    The `len` keyword does not apply on generators.
    This makes the trick without expanding the list!
    """
    return sum(1 for _ in iterable)


# Note how we can pass a range to `plt.plot`, not necessarily a list
# Note the multi-level generator in the second part of the array
interval = range(1, 1000)
plt.plot(interval, [length(syracuse(i)) for i in interval], "r.")

In [ ]:
[i for i in range(1, 50) if length(syracuse(i)) > 100]

27 is the first integer when the Syracuse suite length exceeds 100.  
We can pretty-print the suite with another kind of list comprehension.

In [ ]:
print(" ⇢ ".join(str(i) for i in syracuse(27)))

In [ ]:
plt.plot(list(syracuse(27)))

We can see the suite goes up to 9232 (!) before coming back (quite fast) down to `[4, 2, 1]`.  
Another interesting plot shows the *height* of the suite for each integer.

<div class="alert alert-danger">
<b>Important note:</b> `max` is also a construction that accepts a generator as parameter!
</div>

In [ ]:
ax = plt.axes()
ax.set_yscale("log")
ax.set_yticks([1 << i for i in range(1, 17)])
ax.set_yticklabels([1 << i for i in range(1, 17)])

interval = range(1, 500)
ax.plot(interval, [max(syracuse(i)) for i in interval], "r.")

A last interesting graph plots the height of a suite depending on its length.

In [ ]:
ax = plt.axes()
ax.set_yscale("log")
ax.set_yticks([1 << i for i in range(1, 17)])
ax.set_yticklabels([1 << i for i in range(1, 17)])

interval = range(1, 500)
ax.plot(
    [length(syracuse(i)) for i in interval], [max(syracuse(i)) for i in interval], "r."
)

### Read more about it

- The PEP about list comprehensions: https://www.python.org/dev/peps/pep-0202/
- An interesting story about code optimisation: https://www.python.org/doc/essays/list2str/
- More about this new notation: https://en.wikipedia.org/wiki/Set-builder_notation